In [1]:
from datetime import datetime, timedelta
import pytz
import pandas as pd
from tqdm import tqdm
from modeler.modeler import Modeler as m
from processor.processor import Processor as p
from database.market import Market
from database.adatabase import ADatabase
import numpy as np
import matplotlib.pyplot as plt
import pickle
from functional.backtester import Backtester as b

In [2]:
start_date = datetime(2020,1,1)
end_date = datetime(2023,1,1)

In [3]:
market = Market()
speculation_db = ADatabase("spec")

In [4]:
market.connect()
prices = market.retrieve("prices")
sp500 = market.retrieve("sp500")
market.disconnect()
prices = p.column_date_processing(prices)
prices["year"] = [x.year for x in prices["date"]]
prices["week"] = [x.week for x in prices["date"]]
prices["quarter"] = [x.quarter for x in prices["date"]]
sp500 = sp500.rename(columns={"Symbol":"ticker"})

In [5]:
speculation_db.connect()
simulation = speculation_db.retrieve("sim")
speculation_db.disconnect()

In [6]:
simulation["week"] = simulation["week"] + 1

In [7]:
simulation.sort_values("year")

,year,week,ticker,adjclose,y,0,1,2,3,4,...,12,13,xgb_prediction,xgb_score,cat_prediction,cat_score,tf_prediction,tf_score,prediction,training_years
0,2009,2,MMM,39.056999,38.546269,39.056999,36.795336,37.725406,37.246349,39.975522,...,37.700278,43.473747,39.186550,0.997302,38.299536,0.976151,38.698959,0.996555,38.728349,4
15026,2009,22,NTRS,37.794388,39.980249,37.794388,38.585833,39.316620,39.040769,40.518230,...,41.062675,40.195836,37.782040,0.997302,37.842899,0.976151,39.219627,0.996555,38.281522,4
15025,2009,21,NTRS,38.585833,37.794388,38.585833,39.316620,39.040769,40.518230,45.931251,...,40.195836,42.917674,38.501507,0.997302,38.322821,0.976151,39.503735,0.996555,38.776021,4
15024,2009,20,NTRS,39.316620,38.585833,39.316620,39.040769,40.518230,45.931251,43.114256,...,42.917674,43.028228,39.036633,0.997302,39.976036,0.976151,40.404327,0.996555,39.805665,4
15023,2009,19,NTRS,39.040769,39.316620,39.040769,40.518230,45.931251,43.114256,43.431267,...,43.028228,41.191893,38.697319,0.997302,38.070468,0.976151,41.488739,0.996555,39.418842,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
323890,2022,7,EA,134.357138,131.429933,134.357138,134.184131,131.081929,136.394947,129.950421,...,136.769926,140.427981,133.773407,0.997573,136.573206,0.916578,135.693558,0.997770,135.346724,4
323889,2022,6,EA,134.184131,134.357138,134.184131,131.081929,136.394947,129.950421,132.436160,...,140.427981,139.707094,133.773407,0.997573,136.573206,0.916578,135.393234,0.997770,135.246616,4
323888,2022,5,EA,131.081929,134.184131,131.081929,136.394947,129.950421,132.436160,132.442125,...,139.707094,140.094347,132.317841,0.997573,131.392870,0.916578,132.375275,0.997770,132.028662,4
323898,2022,15,EA,124.580628,122.559909,124.580628,126.707386,125.657947,122.505645,123.520289,...,129.950421,132.436160,124.507805,0.997573,125.490829,0.916578,127.166115,0.997770,125.721583,4


In [8]:
simulation = prices.merge(simulation.drop("adjclose",axis=1),on=["year","week","ticker"],how="left")

In [9]:
sim = simulation.dropna()

In [10]:
sim["delta"] = (sim["prediction"] - sim["adjclose"]) / sim["adjclose"]

In [11]:
trades = []
reqs = [0.15]
signals = [0.05,0.1,0.15]
values = [True,False]
parameters = []
positions = 10
training_year = 4
for value in values:
    for signal in signals:
        for req in reqs:
            parameter = {"value":value,"signal":signal,"req":req,"training_years":training_year}
            parameters.append(parameter)

In [12]:
speculation_db.connect()
simulation = speculation_db.retrieve("sim")
for parameter in tqdm(parameters):
    current_sim = sim[sim["training_years"]==parameter["training_years"]].reset_index()
    iteration_trades = b.speculation_backtest(sim,parameter,start_date,end_date)
    speculation_db.store("trades",pd.DataFrame(iteration_trades))
speculation_db.disconnect()

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 6/6 [36:08<00:00, 361.35s/it]
